Team Members
- Yutian Wang(#85994168): Yutian contributed to data cleaning, formatting/standardization, visualization, time-series analysis using linear regression, and k-fold cross-validation.
- Ronin Cunningham(#43949676): Ronin contributed to implementing the PyTorch neural network to further analyze the weather and corn prices.
- Prayus Shrestha(#55823454): Prayus contributed to gathering the data through scraping and various APIs.
- Ebin Tomy(#44912301): Ebin contributed by writing the formal analysis and explaining each step.

# Weather Impacts on US Corn Prices

TO-DO: opening paragraph.  This study focuses on the relationship between weather and corn prices in US...

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.neural_network import MLPRegressor
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from rnn_model import RNNModel
from sklearn.metrics import mean_absolute_error, mean_squared_error
from gru_model import GRUModel
import shap

%matplotlib inline

TO-DO: incorporate Prayus' data scrapping section here

TO-DO: add description for these two data sets. source? meaning of each column? 

### Data Cleaning and Formatting

In [ ]:
weather_df = pd.read_csv('../data/weather_data.csv', index_col=0)
weather_df.head()

In [ ]:
weather_df.index = pd.to_datetime(weather_df["time"])
weather_df.index.name = "date"
weather_df["date"] = pd.to_datetime(weather_df.index)
weather_df = weather_df[["temperature_2m_mean", "precipitation_sum"]]
weather_df.head()

In [ ]:
oil_df = pd.read_csv("../data/crude_oil.csv", index_col=0)
lumber_df = pd.read_csv("../data/lumber_data.csv", index_col=0)
oat_df = pd.read_csv("../data/oat_data.csv", index_col=0)
wheat_df = pd.read_csv("../data/wheat_data.csv", index_col=0)

In [ ]:
corn_df = pd.read_csv("../data/corn_data.csv", index_col = 0)
corn_df.head()

In [ ]:
def clean_commodity_df(df: pd.DataFrame, price_col_name: str = "price") -> pd.DataFrame: 
    """Cleans the Commodity DataFrame"""
    
    # Setting the index to be the date
    df.index = pd.to_datetime(df.index)
    df.index.name = "date"
    
    # Setting price to be the day's midpoint, and dropping unnecessary columns
    df[price_col_name] = (df["high"] + df["low"]) / 2 
    df = df[[price_col_name]]
    df = df.dropna() 

    # Sorting by date
    df = df.sort_index()
    return df

In [ ]:
corn_df = clean_commodity_df(df=corn_df, price_col_name="corn_price")
corn_df.head()

In [ ]:
oil_df = clean_commodity_df(df=oil_df, price_col_name="oil_price")
lumber_df = clean_commodity_df(df=lumber_df, price_col_name="lumber_price")
oat_df = clean_commodity_df(df=oat_df, price_col_name="oat_price")
wheat_df = clean_commodity_df(df=wheat_df, price_col_name="wheat_price")

In [ ]:
# Merging features
X = pd.concat([oil_df, lumber_df, oat_df, wheat_df, weather_df], axis=1).dropna()
X.head()

In [ ]:
# Lag variables 
lags = [1, 7, 30, 84, 365] # day, week, month, 84 days = time taken for corn to grow, year

# Creating lags in df
for lag in lags: 
    X[f"temp_lag_{lag}"] = X["temperature_2m_mean"].shift(lag)
    X[f"precip_lag_{lag}"] = X["precipitation_sum"].shift(lag)

X.dropna(inplace=True)
X.head()

In [ ]:
# Autoregressive 
X = X.merge(corn_df, on="date")
X["AR1_corn_price"] = X["corn_price"].shift(1)
X.drop("corn_price", axis=1, inplace=True)
X.dropna(inplace=True)
X.head(3)

In [ ]:
y = X.join(corn_df, how="inner")["corn_price"]

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Scaling inputs based on training data
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
mlp = MLPRegressor((100, 100), random_state=123, max_iter=1000000)
mlp.fit(X_train_scaled, y_train)
# mlp.predict(X_test)
# mlp.predict(X_test)
# mse_nn = mean_squared_error(y_test, mlp.predict(X_test))
# print(mse_nn)

It's important to consider the time lag between the weather and corn price data. We should account for the fact that weather conditions in a given year may affect the corn harvest and therefore corn prices mse_nnthe following year. So we shift the index of corn_data backwards by 365 days and create a new column called 'shifted_price_date' with the shifted index values, then to merge the two dataframes.

In [ ]:
mse_nn = mean_squared_error(y_test, mlp.predict(X_test_scaled))
print(mse_nn)

In [ ]:
# corn_data['shifted_price_date'] = corn_data.index - pd.DateOffset(years=1)
# corn_data = corn_data.reset_index()
# merged_data = pd.merge(corn_data, weather_data, left_on='shifted_price_date', right_on='weather_date')
# merged_data = merged_data[['corn_price_date', 'avg_price', 'temperature_2m_mean', 'precipitation_sum']]
# merged_data = merged_data.set_index('corn_price_date')
# merged_data.head()

### Data Standardization

As temperature, precipitation, and price are on different scales, we standardize them to make it easier for comparison. Variables that have large values, such as temperature or precipitation, can dominate the analysis if they are not standardized.

In [ ]:
# # Standardize the data
# scaler = StandardScaler()
# merged_data_scaled = pd.DataFrame(scaler.fit_transform(merged_data), columns=merged_data.columns, index=merged_data.index)
# merged_data_scaled.head()

### Trial Plotting

In [ ]:
# # plot the time-series
# merged_data_scaled.plot(y='avg_price', figsize=(10,5))
# plt.xlabel('Date')
# plt.ylabel('Corn Average Price')
# plt.title('Corn Average Price Overview')

In [ ]:
# plt.scatter(merged_data_scaled['temperature_2m_mean'], merged_data_scaled['avg_price'])
# plt.xlabel('Temperature (°C)')
# plt.ylabel('Avg. Corn Price ($/bushel)')
# plt.title('Temperature vs. Avg. Corn Price')
# plt.show()

# plt.scatter(merged_data_scaled['precipitation_sum'], merged_data_scaled['avg_price'])
# plt.xlabel('Precipitation (mm)')
# plt.ylabel('Avg. Corn Price ($/bushel)')
# plt.title('Precipitation vs. Avg. Corn Price')
# plt.show()

# plt.scatter(merged_data_scaled['temperature_2m_mean'], merged_data_scaled['precipitation_sum'])
# plt.xlabel('Temperature (°C)')
# plt.ylabel('Precipitation (mm)')
# plt.title('Temperature vs. Precipitation')
# plt.show()

Based on the plots, there doesn't seem to be a strong correlation between temperature and precipitation with corn prices. It's also possible that the relationship between weather and corn prices is more complex than a simple linear relationship, and may require further analysis to uncover.

In [ ]:
# corr_matrix = merged_data_scaled[['avg_price', 'temperature_2m_mean', 'precipitation_sum']].corr()
# print(corr_matrix)

In [ ]:
# sns.heatmap(corr_matrix, cmap='coolwarm', annot=True)

### Time Series

Using lags of 30 days and 60 days to capture any possible autocorrelation, without overfitting the model or introducing too much noise. 

In [ ]:
# # create lagged variables
# merged_data_scaled['temp_lag1'] = merged_data_scaled['temperature_2m_mean'].shift(30)
# merged_data_scaled['temp_lag2'] = merged_data_scaled['temperature_2m_mean'].shift(60)
# merged_data_scaled['precip_lag1'] = merged_data_scaled['precipitation_sum'].shift(30)
# merged_data_scaled['precip_lag2'] = merged_data_scaled['precipitation_sum'].shift(60)
# merged_data_scaled = merged_data_scaled.dropna()

In [ ]:
# # Plot the merged data
# fig, axs = plt.subplots(4, figsize=(12, 12))
# axs[0].plot(merged_data_scaled.index, merged_data_scaled['avg_price'], label='Corn Price')
# axs[0].legend()
# axs[1].plot(merged_data_scaled.index, merged_data_scaled['temperature_2m_mean'], label='Temperature')
# axs[1].legend()
# axs[2].plot(merged_data_scaled.index, merged_data_scaled['precipitation_sum'], label='Precipitation')
# axs[2].legend()
# axs[3].plot(merged_data_scaled.index, merged_data_scaled['temp_lag1'], label='Temperature Lag 1')
# axs[3].plot(merged_data_scaled.index, merged_data_scaled['temp_lag2'], label='Temperature Lag 2')
# axs[3].plot(merged_data_scaled.index, merged_data_scaled['precip_lag1'], label='Precipitation Lag 1')
# axs[3].plot(merged_data_scaled.index, merged_data_scaled['precip_lag2'], label='Precipitation Lag 2')
# axs[3].legend()
# plt.show()

### Linear Regression Analysis

In [ ]:
# X = merged_data_scaled[['temp_lag1', 'temp_lag2', 'precip_lag1', 'precip_lag2']]
# y = merged_data_scaled['avg_price']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# lr_model = LinearRegression()
# lr_model.fit(X_train, y_train)

# print('Coefficients:', lr_model.coef_)
# print('Intercept:', lr_model.intercept_)

In [ ]:
# # make predictions on test data
# y_pred = lr_model.predict(X_test)

# train_rmse = np.sqrt(np.mean((lr_model.predict(X_train) - y_train) ** 2))
# test_rmse = np.sqrt(np.mean((y_pred - y_test) ** 2))

# print(f'Train RMSE: {train_rmse}')
# print(f'Test RMSE: {test_rmse}')

TO-DO: add some discussions here

### Cross-Validation

In [ ]:
# lr2 = LinearRegression()
# X = merged_data_scaled.drop('avg_price', axis=1)
# y = merged_data_scaled['avg_price']

# mse_scores = -cross_val_score(lr2, X, y, cv=5, scoring='neg_mean_squared_error')

# print(f"Average MSE: {mse_scores.mean():.4f}, Standard deviation: {mse_scores.std():.4f}")

### RNN Regression

TODO: explanation

In [ ]:
pd.DataFrame(X_train_scaled, columns=X_train.columns)

In [ ]:
y_train

#### Basic RNN model with Adam optimizer and L2 regularization

##### Training

First, we need to set hyperparameters for our RNN model. After experimenting with various different configurations, below are the hyperparameters that I found to be best

In [ ]:
# Set hyperparameters
input_size = 17
hidden_size = 64
output_size = 1
num_epochs = 1000
batch_size = 32
learning_rate = 0.001
sequence_length = 1

# Prepare the train dataset
X_train_arr = np.array(X_train_scaled)
y_train_arr = np.array(y_train)

X_train_arr = X_train_arr.reshape(-1, sequence_length, input_size)
y_train_arr = y_train_arr.reshape(-1, sequence_length, output_size)

train_dataset = TensorDataset(torch.tensor(X_train_arr).float(), torch.tensor(y_train_arr).float())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
rnn_model_instance = RNNModel(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(rnn_model_instance.parameters(), lr=learning_rate, weight_decay=1e-5)  # Add weight_decay for L2 regularization

# Train the model
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        # Forward pass
        outputs = rnn_model_instance(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()



##### Testing

In [ ]:
# Prepare the test dataset
X_test_arr = np.array(X_test_scaled)
y_test_arr = np.array(y_test)

X_test_arr = X_test_arr.reshape(-1, sequence_length, input_size)
y_test_arr = y_test_arr.reshape(-1, sequence_length, output_size)

test_dataset = TensorDataset(torch.tensor(X_test_arr).float(), torch.tensor(y_test_arr).float())
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Evaluate the model
rnn_model_instance.eval()
predictions = []
actuals = []

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = rnn_model_instance(inputs)
        predictions.extend(outputs.numpy().flatten())
        actuals.extend(targets.numpy().flatten())

# Calculate accuracy metrics
mae_rnn = mean_absolute_error(actuals, predictions)
mse_rnn = mean_squared_error(actuals, predictions)
rmse_rnn = np.sqrt(mse_rnn)

print(f'Mean Absolute Error: {mae_rnn:.4f}')
print(f'Mean Squared Error: {mse_rnn:.4f}')
print(f'Root Mean Squared Error: {rmse_rnn:.4f}')

#### GRU model with Adam optimizer and L2 Regularization

##### Training

In [ ]:
# Set hyperparameters

input_size = 17
hidden_size = 64
output_size = 1
num_epochs = 1000
batch_size = 32
learning_rate = 0.001
sequence_length = 1

# Prepare the train dataset
X_train_arr = np.array(X_train_scaled)
y_train_arr = np.array(y_train)

X_train_arr = X_train_arr.reshape(-1, sequence_length, input_size)
y_train_arr = y_train_arr.reshape(-1, sequence_length, output_size)

train_dataset = TensorDataset(torch.tensor(X_train_arr).float(), torch.tensor(y_train_arr).float())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
gru_model_instance = GRUModel(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(gru_model_instance.parameters(), lr=learning_rate, weight_decay=1e-5)

# Train the modelt
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        # Forward pass
        outputs = gru_model_instance(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


##### Testing

In [ ]:
# Prepare the test dataset
X_test_arr = np.array(X_test_scaled)
y_test_arr = np.array(y_test)

X_test_arr = X_test_arr.reshape(-1, sequence_length, input_size)
y_test_arr = y_test_arr.reshape(-1, sequence_length, output_size)

test_dataset = TensorDataset(torch.tensor(X_test_arr).float(), torch.tensor(y_test_arr).float())
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Evaluate the model
gru_model_instance.eval()
predictions = []
actuals = []

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = gru_model_instance(inputs)
        predictions.extend(outputs.numpy().flatten())
        actuals.extend(targets.numpy().flatten())

# Calculate accuracy metrics
mae_gru = mean_absolute_error(actuals, predictions)
mse_gru = mean_squared_error(actuals, predictions)
rmse_gru = np.sqrt(mse_gru)

print(f'Mean Absolute Error: {mae_gru:.4f}')
print(f'Mean Squared Error: {mse_gru:.4f}')
print(f'Root Mean Squared Error: {rmse_gru:.4f}')

### Shapley Values of our Best Model

In [ ]:
best_model = rnn_model_instance if rmse_rnn <= rmse_gru else gru_model_instance

def model_wrapper(x):
    x_tensor = torch.FloatTensor(x).unsqueeze(1)
    with torch.no_grad():
        output = best_model(x_tensor)
    return output.numpy()

# Initialize the explainer
explainer = shap.Explainer(model_wrapper, X_train_scaled)

# Compute SHAP values for X_test
shap_values = explainer(X_test_scaled)

# Plot SHAP summary plot
shap.summary_plot(shap_values, X_test_scaled, feature_names=X.columns)